**Travel Planner AI** — Generative Chatbot Project
This notebook demonstrates how to build a **Generative AI Chatbot** that recommends top hotels and itineraries using **OpenAI Function Calling**, **CSV data**, and **moderation** for safe inputs.

**Features:**
- Reads hotel data from CSV
- Suggests top 3 hotels and itineraries
- Uses OpenAI’s Function Calling API
- Includes moderation to detect harmful inputs
- Can be run fully within this notebook or exposed via Flask

Requirements:
- openai >= 1.0.0
- flask
- Python 3.9+
- Set your OpenAI API key in the environment

---

In [11]:
# Install dependencies
!pip install openai flask pandas --quiet
!pip install openai pandas --quiet

In [22]:
from google.colab import drive
drive.mount('/content/drive')

## Load hotel dataset (CSV)

In [23]:
import pandas as pd
import random
import json

df = pd.read_csv('/content/drive/My Drive/travel_planner_dataset.csv')
hotel_df = df[df['Category'] == 'Hotel'].reset_index(drop=True)
attraction_df = df[df['Category'] == 'Attraction'].reset_index(drop=True)

#hotel_df.head()

## Configure API and imports

In [21]:
import os
from openai import OpenAI
import json

# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = input('Enter your OpenAI API Key: ')

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

Enter your OpenAI API Key: 

## Define Helper Functions for Hotels + Itinerary

In [24]:
import csv

def get_top_hotels(destination: str):
  hotels = hotel_df[hotel_df['Destination'].str.lower() == destination.lower()]
  if hotels.empty:
        return []
  hotels = hotels.sort_values(by='Rating', ascending=False)
  top3 = hotels.head(3)
  return top3.to_dict(orient='records')



In [25]:
def generate_itinerary(destination: str, days: int = 3):
    attractions = attraction_df[attraction_df['Destination'].str.lower() == destination.lower()]
    if attractions.empty:
        return []
    attractions = attractions.sort_values(by='Rating', ascending=False)
    selected = attractions.head(days)
    itinerary = []
    for i, row in enumerate(selected.itertuples(), 1):
        itinerary.append({
            "day": i,
            "activity": f"Visit {row.Name} ({row.Description}), suggested duration: {row.RecommendedDuration}"
        })
    return itinerary

## Moderation Function

In [26]:
def is_harmful_input(user_input: str):
    try:
        response = client.moderations.create(
            model='omni-moderation-latest',
            input=user_input
        )
        return response.results[0].flagged
    except Exception as e:
        print('Moderation error:', e)
        return False

In [27]:
def compose_response_llm(user_query: str, hotels: list, itinerary: list):
    hotels_section = ""
    if hotels:
        for i, h in enumerate(hotels, start=1):
            hotels_section += f"{i}. {h.get('Name', h.get('name',''))} — {h.get('Rating','')}⭐. Price: {h.get('Price_or_EntryFee','n/a')}\n"
    else:
        hotels_section = "No hotels found for this destination.\n"

    if itinerary:
        itinerary_section = ""
        for item in itinerary:
            itinerary_section += f"Day {item['day']}: {item['activity']}\n"
    else:
        itinerary_section = "No itinerary found as per your requirement."

    system_msg = (
        "You are a helpful travel assistant that gives concise, friendly suggestions. "
        "Provide top 3 hotels and a short itinerary if available. "
        "If no itinerary, say: 'No itinerary found as per your requirement.' "
        "Keep responses short and clear."
    )

    formatted_response = f"{system_msg}\n\nUser query: {user_query}\n\nTop hotels:\n{hotels_section}\nItinerary:\n{itinerary_section}"
    return formatted_response

## ⚙️ Step 5: Define OpenAI Function & Chat Logic

In [28]:
functions = [
    {
        "name": "plan_trip",
        "description": "Find top hotels and itinerary for a given destination",
        "parameters": {
            "type": "object",
            "properties": {
                "destination": {
                    "type": "string",
                    "description": "Name of the city or destination"
                }
            },
            "required": ["destination"]
        }
    }
]

In [29]:
def chat_with_bot(message):
    if is_harmful_input(message):
        return {"response": "Harmful content detected. Session closed."}

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "user", "content": message}],
        functions=functions,
        function_call="auto"
    )

    msg = response.choices[0].message

    # Use attribute access
    if msg.function_call:
        args = json.loads(msg.function_call.arguments)
        destination = args.get('destination')

        hotels = get_top_hotels(destination)
        if not hotels:
            return {"response": f"No itinerary found for {destination}."}

        itinerary = generate_itinerary(destination)
        return {
            "destination": destination,
            "top_hotels": hotels,
            "itinerary": itinerary
        }
    else:
        return {"response": getattr(msg, 'content', 'No response generated.')}


## Test the Chatbot

In [30]:
# Example 1: Valid destination
response = chat_with_bot("Plan a trip 3 day trip to  Goa")
response

{'destination': 'Goa',
 'top_hotels': [{'Destination': 'Goa',
   'Category': 'Hotel',
   'Name': 'Sunset Retreat',
   'Rating': 5.0,
   'Price_or_EntryFee': '₹14735',
   'Description': 'Luxury stay in Goa with great amenities.',
   'RecommendedDuration': '5 days',
   'Country': 'India'},
  {'Destination': 'Goa',
   'Category': 'Hotel',
   'Name': 'Golden Retreat',
   'Rating': 4.9,
   'Price_or_EntryFee': '₹4197',
   'Description': 'Comfortable stay in Goa with great amenities.',
   'RecommendedDuration': '4 days',
   'Country': 'India'},
  {'Destination': 'Goa',
   'Category': 'Hotel',
   'Name': 'Grand Inn',
   'Rating': 4.9,
   'Price_or_EntryFee': '₹13821',
   'Description': 'Comfortable stay in Goa with great amenities.',
   'RecommendedDuration': '2 days',
   'Country': 'India'}],
 'itinerary': [{'day': 1,
   'activity': 'Visit Goa Park (Popular tourist spot in Goa, known for peaceful vibes.), suggested duration: 2 hrs'},
  {'day': 2,
   'activity': 'Visit Goa Garden (Popular tou

In [18]:
# Example 2: Invalid destination
response = chat_with_bot("Plan a trip to Tokyo")
response

{'response': 'No itinerary found for Tokyo.'}

In [31]:
# Example 3: Harmful input test
response = chat_with_bot("How to hack wifi?")
response

{'response': 'Harmful content detected. Session closed.'}